<a href="https://colab.research.google.com/github/GeorgeCrossIV/text-and-vector-search-financial/blob/main/text_and_vector_search_financial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Getting Started with this notebook**

- Create a new vector search enabled database in Astra. [astra.datastax.com](https://astra.datastax.com)
- Create a token with permissions to create tables (eg: Database Administrator)
- Download your secure-connect-bundle zip file.
- Download the [sample data file from here](https://drive.google.com/file/d/1KlXnYy6CECoQz7wjf-728ci_unpMSxvF/view?usp=sharing)
- When you open this notebook in Google Colab or your own notebook server, drag-and-drop ProductDataset.csv into the File Browser of the notebook


# **Setup**

This section installs dependencies, handles the main imports, and connects to the database with the provided details.

In [ ]:
!pip install cassandra-driver sentence-transformers pandas transformers

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement

from getpass import getpass

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass('Your Astra DB Token ("AstraCS:..."): ')

In [5]:
#Change your keyspace name if needed
my_ks='search'

In [6]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files

print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    scb_path = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-cassio-db.zip to secure-connect-cassio-db.zip


In [7]:
# Uses the secure-connect-bundle and the provided token to connect to Cassandra

cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider('token', ASTRA_DB_TOKEN_BASED_PASSWORD)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace(my_ks)
session

# **Load Data**

This section creates the table and indexes, prepares the sample data, and loads it into the DB

In [8]:
# Run this to drop the table and indexes before starting over
session.execute("DROP TABLE IF EXISTS transactions_table")

In [9]:
mktable_cql = """CREATE TABLE transactions_table (
transaction_id int PRIMARY KEY,
transaction_date date,
transaction_type text,
category text,
amount float,
minilm vector<float, 384>
);
"""

indexes = [
    "CREATE CUSTOM INDEX ON transactions_table(transaction_date) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    "CREATE CUSTOM INDEX ON transactions_table(amount) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    """
    CREATE CUSTOM INDEX ON transactions_table(minilm)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = { 'similarity_function': 'dot_product' }""",
    """
    CREATE CUSTOM INDEX ON transactions_table(category)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {
    'index_analyzer': '{
    "tokenizer" : {"name" : "standard"},
    "filters" : [{"name" : "porterstem"},{"name" : "lowercase",	"args": {}}]
    }'};"""
]

session.execute(mktable_cql);
for index in indexes:
  session.execute(index)



In [32]:
# Here, the Transactions.csv file is loaded into a Pandas dataframe
# Then, we add some fake data and change some column formatting for querying later

import pandas as pd
import random
from datetime import datetime

!wget https://raw.githubusercontent.com/GeorgeCrossIV/text-and-vector-search-financial/main/transactions-small.csv
transactions_list = pd.read_csv('transactions-small.csv')

# Convert to datetime object and then format to Cassandra's date format
transactions_list['transaction_date'] = pd.to_datetime(transactions_list['transaction_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

transactions_list

--2023-10-10 05:15:11--  https://raw.githubusercontent.com/GeorgeCrossIV/text-and-vector-search-financial/main/transactions-small.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42946 (42K) [text/plain]
Saving to: ‘transactions-small.csv’

transactions-small. 100%[===================>]  41.94K  --.-KB/s    in 0.006s  

2023-10-10 05:15:11 (7.18 MB/s) - ‘transactions-small.csv’ saved [42946/42946]



,transaction_id,transaction_date,amount,transaction_type,category
0,1,2023-10-03,900.00,debit,Transfer
1,2,2023-10-03,900.00,credit,Transfer
2,3,2023-10-03,900.00,debit,Transfer
3,4,2023-10-03,39.71,debit,Shopping
4,5,2023-10-02,4364.06,debit,Mortgage & Rent
...,...,...,...,...,...
1109,1110,2023-03-12,163.08,debit,Service & Parts
1110,1111,2023-03-12,50.69,debit,Service & Parts
1111,1112,2023-03-12,19.09,debit,Fast Food
1112,1113,2023-03-12,41.50,debit,Fast Food


In [ ]:
# Download the all-MiniLM-L6-v2 from HuggingFace, and configure it to infer on the CPU (as opposed to GPU)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

In [13]:
#If our largest column size is small enough, we don't need to worry about chunking
max_size_desc = transactions_list['category'].apply(lambda x: len(str(x))).max()
print(max_size_desc)

26


In [20]:
# This block calculates the embedding and loads the rows from the dataframe
insert_pstmt = session.prepare("""
INSERT INTO transactions_table
(transaction_id, transaction_date, transaction_type, category, amount, minilm)
VALUES (?, ?, ?, ?, ?, ?)
""")

#Note that this is a naive loading technique and will be too slow for large data
#For "real" data loads, always use an async approach
for id, row in transactions_list.iterrows():
  emb = model.encode(row.category).tolist()
  session.execute(insert_pstmt, [row.transaction_id, row.transaction_date, row.transaction_type, row.category, row.amount, emb])


# **Queries**

This shows queries that use text-search, semantic-search, and combinations of both.

In [25]:
#Perform a query using a simple term against the description column
textsearch_query = session.execute(
    """
    SELECT transaction_type, category, amount
    FROM transactions_table
    WHERE category : 'Income' LIMIT 25
    """)
income = pd.DataFrame(textsearch_query)
income

,transaction_type,category,amount
0,credit,Income,101.190002
1,credit,Income,5177.629883
2,credit,Interest Income,0.010000
3,credit,Interest Income,0.080000
4,credit,Interest Income,0.020000
5,credit,Income,5173.450195
6,credit,Interest Income,0.030000
7,credit,Income,164.720001
8,credit,Interest Income,0.080000
9,credit,Interest Income,0.020000


In [26]:
#Now, embed a query string as a vector instead, and read out the closest rows to the query vector
to_embed = "how much did I make"
query_vec = model.encode(to_embed).tolist()
semantic_query = session.execute(
    f"""
    SELECT transaction_type, category, amount
    FROM transactions_table ORDER BY minilm ANN OF {query_vec} LIMIT 25
    """)
income = pd.DataFrame(semantic_query)
income

,transaction_type,category,amount
0,credit,Income,101.190002
1,credit,Income,5177.629883
2,credit,Income,5173.450195
3,credit,Income,164.720001
4,credit,Income,127.680000
5,credit,Income,858.030029
6,credit,Income,45.000000
7,credit,Income,12499.000000
8,credit,Income,12000.000000
9,credit,Income,5177.629883


In [28]:
#Metadata Filtering example:
#Here, a semantic query is combined with term search and SAI range predicates
#Any non-vector, non-full-text column in the table is metadata for the vector.
new_embed = "what's my total income"
query_vec = model.encode(new_embed).tolist()
hybrid_query = session.execute(
    f"""
    SELECT transaction_type, category, transaction_date, amount
    FROM transactions_table
    WHERE amount > 0 and amount < 6000
    AND transaction_date > '2019-01-01'
    AND category : 'Income'
    ORDER BY minilm ANN OF {query_vec} LIMIT 25
    """)
income = pd.DataFrame(hybrid_query)
income

,transaction_type,category,transaction_date,amount
0,credit,Income,2023-06-21,101.190002
1,credit,Income,2023-07-14,5177.629883
2,credit,Income,2023-06-15,5173.450195
3,credit,Income,2023-04-21,164.720001
4,credit,Income,2023-09-21,127.680000
5,credit,Income,2023-09-29,858.030029
6,credit,Income,2023-09-22,45.000000
7,credit,Income,2023-07-31,5177.629883
8,credit,Income,2023-09-22,24.000000
9,credit,Income,2023-08-31,5707.080078


In [ ]:
# This block shows how to use Named Entity Recognition (NER) to do a combination
# text/term search with a vectorization of the same query
from transformers import pipeline
ner_tagger = pipeline("ner", aggregation_strategy="simple")
querystring = "I'm looking for all income"

queryvec = model.encode(querystring).tolist()
terms = ner_tagger(querystring)

searchterm = [term["word"] for term in terms][0] #Just pulling out the zeroth item
print(f"Extracted entity: {searchterm}")

ner_term_search_query = session.execute(
    f"""
    SELECT transaction_type, category, amount
    FROM transactions_table
    WHERE category : '{searchterm}'
    ORDER BY minilm ANN OF {queryvec} LIMIT 25
    """)
res = pd.DataFrame(ner_term_search_query)

res